In [2]:
!pip install web3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.24.4 which is incompatible.


  Using cached cached_property-1.5.2-py2.py3-none-any.whl (7.6 kB)
  Using cached jsonschema-4.17.3-py3-none-any.whl (90 kB)
  Using cached protobuf-4.24.4-cp37-cp37m-win_amd64.whl (430 kB)
  Created wheel for parsimonious: filename=parsimonious-0.9.0-py3-none-any.whl size=44301 sha256=a8b6b85c171991d938e9bfa9aad83188f1c6ad4ba37d8f2e4779c1562b7902ec
  Stored in directory: c:\users\nexus computers\appdata\local\pip\cache\wheels\a4\54\88\c1ee7de0eabd1fb817cbf35824e4c2cba664d5816ddc64efb1
Successfully built parsimonious
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 3.2.0
    Uninstalling jsonschema-3.2.0:
      Successfully uninstalled jsonschema-3.2.0


In [3]:
import os
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from web3 import Web3

In [9]:
# --- Blockchain Setup ---
# Connect to local Ganache blockchain
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
assert w3.is_connected(), "Blockchain connection failed"
print("Connected to Ganache:", w3.is_connected())

Connected to Ganache: True


In [11]:
# Set your deployed contract address and ABI here
contract_address = "0x3858dab8be5218d3a433a536ab6df1b7cfc029db"  # REPLACE with your contract address
abi =[
        {
            "inputs": [],
            "name": "claimReward",
            "outputs": [],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "inputs": [],
            "stateMutability": "nonpayable",
            "type": "constructor"
        },
        {
            "anonymous": False,
            "inputs": [
                {
                    "indexed": True,
                    "internalType": "address",
                    "name": "participant",
                    "type": "address"
                }
            ],
            "name": "ConsentGiven",
            "type": "event"
        },
        {
            "anonymous": False,
            "inputs": [
                {
                    "indexed": True,
                    "internalType": "address",
                    "name": "participant",
                    "type": "address"
                }
            ],
            "name": "ConsentRevoked",
            "type": "event"
        },
        {
            "inputs": [],
            "name": "depositRewardPool",
            "outputs": [],
            "stateMutability": "payable",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "distributeRewards",
            "outputs": [],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "giveConsent",
            "outputs": [],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "revokeConsent",
            "outputs": [],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "anonymous": False,
            "inputs": [
                {
                    "indexed": True,
                    "internalType": "address",
                    "name": "participant",
                    "type": "address"
                },
                {
                    "indexed": False,
                    "internalType": "uint256",
                    "name": "amount",
                    "type": "uint256"
                }
            ],
            "name": "RewardClaimed",
            "type": "event"
        },
        {
            "anonymous": False,
            "inputs": [
                {
                    "indexed": False,
                    "internalType": "uint256",
                    "name": "totalRewardPool",
                    "type": "uint256"
                }
            ],
            "name": "RewardsDistributed",
            "type": "event"
        },
        {
            "inputs": [
                {
                    "internalType": "uint256",
                    "name": "score",
                    "type": "uint256"
                }
            ],
            "name": "submitUpdate",
            "outputs": [],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "anonymous": False,
            "inputs": [
                {
                    "indexed": True,
                    "internalType": "address",
                    "name": "participant",
                    "type": "address"
                },
                {
                    "indexed": False,
                    "internalType": "uint256",
                    "name": "score",
                    "type": "uint256"
                }
            ],
            "name": "UpdateSubmitted",
            "type": "event"
        },
        {
            "inputs": [],
            "name": "owner",
            "outputs": [
                {
                    "internalType": "address",
                    "name": "",
                    "type": "address"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [
                {
                    "internalType": "uint256",
                    "name": "",
                    "type": "uint256"
                }
            ],
            "name": "participantAddresses",
            "outputs": [
                {
                    "internalType": "address",
                    "name": "",
                    "type": "address"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [
                {
                    "internalType": "address",
                    "name": "",
                    "type": "address"
                }
            ],
            "name": "participants",
            "outputs": [
                {
                    "internalType": "bool",
                    "name": "consentGiven",
                    "type": "bool"
                },
                {
                    "internalType": "uint256",
                    "name": "contributionScore",
                    "type": "uint256"
                },
                {
                    "internalType": "uint256",
                    "name": "reward",
                    "type": "uint256"
                },
                {
                    "internalType": "bool",
                    "name": "exists",
                    "type": "bool"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "totalContributionScore",
            "outputs": [
                {
                    "internalType": "uint256",
                    "name": "",
                    "type": "uint256"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "totalRewardPool",
            "outputs": [
                {
                    "internalType": "uint256",
                    "name": "",
                    "type": "uint256"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        }
    ]

# Use Ganache accounts (assumes accounts are unlocked)
owner_account = w3.eth.accounts[0]
client_accounts = w3.eth.accounts[1:6]  # Five client accounts


In [12]:
# --- Federated Learning Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define dataset paths and transforms (update paths as necessary)
base_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
def visualize_images(dataset, num_images=9):
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        plt.subplot(3, 3, i + 1)
        image, label = dataset[i]
        plt.imshow(image.permute(1, 2, 0))
        plt.title(f"Label: {label}")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_images(train_dataset)

In [ ]:
# Define the model architecture using MobileNet V2
def create_model():
    model = models.mobilenet_v2(pretrained=True)
    model.features[18] = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Conv2d(320, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False),
        nn.BatchNorm2d(1280)
    )
    model.classifier[1] = nn.Linear(model.last_channel, 1)
    return model

global_model = create_model().to(device)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Federated averaging: aggregates local models
def federated_averaging(models_list):
    global_model = copy.deepcopy(models_list[0])
    with torch.no_grad():
        for key in global_model.state_dict().keys():
            tensors = [model.state_dict()[key].data.float() for model in models_list]
            global_model.state_dict()[key].data.copy_(torch.mean(torch.stack(tensors), dim=0))
    return global_model

In [ ]:
# Split dataset into subsets for each client
def split_dataset(dataset, num_clients):
    indices = np.arange(len(dataset))
    np.random.shuffle(indices)
    split_indices = np.array_split(indices, num_clients)
    return [Subset(dataset, idx) for idx in split_indices]

num_clients = 5
client_datasets = split_dataset(train_dataset, num_clients)
client_dataloaders = [DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in client_datasets]

# Initialize client models as copies of the global model
client_models = [copy.deepcopy(global_model) for _ in range(num_clients)]

In [ ]:
# --- Smart Contract Interaction: Consent ---
# Each client gives consent by calling the contract's giveConsent() function.
for i, client_account in enumerate(client_accounts):
    tx_hash = contract.functions.giveConsent().transact({'from': client_account})
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Client {i+1} consent given: {receipt.transactionHash.hex()}")

In [ ]:
# --- Training and Contract Submission ---
def train_model(model, dataloader, criterion, optimizer, epochs=1):
    model.train()
    running_loss = 0.0
    for _ in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    avg_loss = running_loss / len(dataloader)
    return model, avg_loss

In [ ]:
# Run federated training over multiple rounds
num_rounds = 10
client_update_scores = [0] * num_clients  # To accumulate each client's performance score

for round_num in range(num_rounds):
    print(f"Round {round_num+1}/{num_rounds}")
    # Each client trains on its local data and submits an update score
    for i, (client_model, dataloader, client_account) in enumerate(zip(client_models, client_dataloaders, client_accounts)):
        optimizer = optim.Adam(client_model.parameters(), lr=0.001)
        client_model, train_loss = train_model(client_model, dataloader, criterion, optimizer)
        # Using inverse of training loss as a performance metric (higher is better)
        performance_score = 1 / (train_loss + 1e-5)
        client_update_scores[i] += performance_score
        
        # Scale score to an integer (e.g., multiply by 1e6)
        scaled_score = int(performance_score * 1e6)
        tx_hash = contract.functions.submitUpdate(scaled_score).transact({'from': client_account})
        receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        print(f"Client {i+1} update submitted with score {performance_score:.4f}: {receipt.transactionHash.hex()}")
    
    # Aggregate client models using federated averaging
    global_model = federated_averaging(client_models)
    # Distribute the updated global model to all clients
    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())

In [ ]:
# --- Reward Distribution ---
# Owner deposits reward pool (e.g., 1 Ether) into the contract
tx_hash = contract.functions.depositRewardPool().transact({
    'from': owner_account,
    'value': w3.toWei(1, 'ether')
})
receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("Owner deposited reward pool:", receipt.transactionHash.hex())

# Owner calls distributeRewards() to allocate rewards based on contributions
tx_hash = contract.functions.distributeRewards().transact({'from': owner_account})
receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("Rewards distributed:", receipt.transactionHash.hex())

In [ ]:
# --- Evaluation ---
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = torch.sigmoid(model(images))
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels.unsqueeze(1)).sum().item()
            total += labels.size(0)
    return correct / total

In [ ]:
accuracy = evaluate_model(global_model, val_loader)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# --- Clients Claim Rewards ---
for i, client_account in enumerate(client_accounts):
    tx_hash = contract.functions.claimReward().transact({'from': client_account})
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Client {i+1} claimed reward: {receipt.transactionHash.hex()}")